In [ ]:
#!pip install plotly panel nbformat

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [ ]:
#Read transaction data
df = pd.read_csv('classified_expenses.csv')
#add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month_name()
df.head()


In [7]:
#When I have an Income I want the category to be the description
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

## Make pie chart

In [3]:
def make_pie_chart(df,year, label):
    #Filter data for expense transactions.
    filtered = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    color_scale = px.colors.qualitative.Set2

    pie_fig = px.pie(filtered, values = 'Amount (EUR)', names = 'Category', title = f'{label} Distribution for {year}', color_discrete_sequence=color_scale)
    pie_fig.update_traces(textposition='inside', direction = "clockwise", hole = 0.2,  textinfo='percent+label')

    total_expense = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]['Amount (EUR)'].sum()
    total_income = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]['Amount (EUR)'].sum()

    if label == 'Expense':
        total_text = '€' + str(round(total_expense,2))

        #Saving rates
        saving_rate = round((total_income - total_expense) / total_income * 100,2)
        saving_rate_text = "Saving rate" + str(saving_rate) + '%'
    else:
        saving_rate = ""
        saving_rate_text = ""
        total_text = '€' + str(round(total_income,2))

        pie_fig.update_layout(uniformtext_minsize=10, 
                        uniformtext_mode='hide',
                        title=dict(text=label+" Breakdown " + str(year) + saving_rate_text),
                        # Add annotations in the center of the donut.
                        annotations=[
                            dict(
                                text=total_text, 
                                # Square unit grid starting at bottom left of page
                                x=0.5, y=0.5, font_size=12,
                                # Hide the arrow that points to the [x,y] coordinate
                                showarrow=False
                            )
                        ]
                    )
    return pie_fig

In [ ]:
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
income_pie_fig_2022

## Make bar charts over months in a year

In [5]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [ ]:
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
income_monthly_2022

## Putting all charts together into tabs for 2022/2023

In [ ]:
# Pie charts
income_pie_fig_2022 = make_pie_chart(df, 2022, 'Income')
expense_pie_fig_2022 = make_pie_chart(df, 2022, 'Expense')  
income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')

# Bar charts
income_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Income')
expense_monthly_2022 = make_monthly_bar_chart(df, 2022, 'Expense')
income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')

# Create tabs
tabs = pn.Tabs(
    ('2022', pn.Column(
        pn.Row(
            pn.pane.Plotly(income_pie_fig_2022), 
            pn.pane.Plotly(expense_pie_fig_2022)
        ),
        pn.Row(
            pn.pane.Plotly(income_monthly_2022), 
            pn.pane.Plotly(expense_monthly_2022)
        )
    )),
    ('2023', pn.Column(
        pn.Row(
            pn.pane.Plotly(income_pie_fig_2023), 
            pn.pane.Plotly(expense_pie_fig_2023)
        ),
        pn.Row(
            pn.pane.Plotly(income_monthly_2023), 
            pn.pane.Plotly(expense_monthly_2023)
        )
    ))
)
tabs.show()

## Create a Dashboard

In [ ]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()